In [1]:
  import pandas as pd
  from nltk.sentiment import SentimentIntensityAnalyzer
  import nltk

  # Ensure you download the VADER lexicon
  nltk.download('vader_lexicon')

  # Load your dataset (update the file path accordingly)
  file_path = '/content/news_data_cleaned.csv'
  data = pd.read_csv(file_path)

  # Initialize the VADER sentiment analyzer
  sia = SentimentIntensityAnalyzer()

  # Apply sentiment analysis on the 'Cleaned Article Text' column
  data['Sentiment Score'] = data['Cleaned Article Text'].apply(lambda text: sia.polarity_scores(str(text))['compound'])

  # Categorize the sentiment into Positive, Neutral, or Negative
  data['Sentiment'] = data['Sentiment Score'].apply(
      lambda score: 'Positive' if score > 0.05 else 'Negative' if score < -0.05 else 'Neutral'
  )

  # Save the results to a new CSV file
  output_path = 'sentiment_analysis_results.csv'
  data.to_csv(output_path, index=False)

  print(f"Sentiment analysis completed. Results saved to {output_path}")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Sentiment analysis completed. Results saved to sentiment_analysis_results.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
file_path = '/content/sentiment_analysis_results.csv'
data = pd.read_csv(file_path)

# Ensure there are no missing values in the required columns
data = data.dropna(subset=['Cleaned Article Text', 'Sentiment'])

# Encode sentiment labels
data['Sentiment Label'] = data['Sentiment'].map({'Positive': 1, 'Neutral': 0, 'Negative': -1})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data['Cleaned Article Text'], data['Sentiment Label'], test_size=0.2, random_state=42
)

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Initialize classification models
models = {
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": MultinomialNB(),
    "Support Vector Machine": SVC(),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"\n{model_name} Results")
    # Train the model
    model.fit(X_train_vectorized, y_train)

    # Predictions on training data
    y_train_pred = model.predict(X_train_vectorized)
    print("\nTraining Performance:")
    print(classification_report(y_train, y_train_pred, zero_division=0))

    # Predictions on testing data
    y_test_pred = model.predict(X_test_vectorized)
    print("\nTesting Performance:")
    print(classification_report(y_test, y_test_pred, zero_division=0))


Logistic Regression Results

Training Performance:
              precision    recall  f1-score   support

          -1       1.00      0.92      0.96        25
           0       1.00      0.85      0.92        13
           1       0.89      1.00      0.94        34

    accuracy                           0.94        72
   macro avg       0.96      0.92      0.94        72
weighted avg       0.95      0.94      0.94        72


Testing Performance:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         6
           0       1.00      0.67      0.80         3
           1       0.56      1.00      0.72         9

    accuracy                           0.61        18
   macro avg       0.52      0.56      0.51        18
weighted avg       0.45      0.61      0.49        18


Naive Bayes Results

Training Performance:
              precision    recall  f1-score   support

          -1       1.00      0.52      0.68        25
           

In [3]:
import joblib

# Assuming models dictionary from your code
models = {
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": MultinomialNB(),
    "Support Vector Machine": SVC(),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Save each model to a separate file
for model_name, model in models.items():
    model_save_path = f'/content/{model_name.replace(" ", "_").lower()}_model.pkl'
    joblib.dump(model, model_save_path)
    print(f"{model_name} saved to: {model_save_path}")


Logistic Regression saved to: /content/logistic_regression_model.pkl
Naive Bayes saved to: /content/naive_bayes_model.pkl
Support Vector Machine saved to: /content/support_vector_machine_model.pkl
Random Forest saved to: /content/random_forest_model.pkl
Gradient Boosting saved to: /content/gradient_boosting_model.pkl
